In [96]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import copy
import numpy as np

In [97]:
# Set the tournament name with the year and city of the regional

tournament = 'pokemon-bologna-2025'

homepage = 'https://rk9.gg/event/' + tournament

roster = pd.read_csv('Roster_with_Archetypes_from_'+tournament+'.csv',index_col = [0])
archetype_dict = dict(zip(roster['Name plus country'], roster['Archetype']))

elos = pd.read_csv('Elo_Ratings/updated_for_' + tournament +'.csv', index_col=[0])
elo_dict = dict(zip(elos['Name'], elos['Rating']))

In [98]:
def get_rk9_urls(homepage):
    soup = BeautifulSoup(requests.get(homepage).text)

    tcg = soup.find("div", class_ = 'card h-100 mt-3 p-2 shadow bg-blue-050') # Locate tcg box (sometimes indigo)

    # Find url extensions for roster and pairings
    roster_code = tcg.find('a', {'href': re.compile('/roster*')})['href']  
    pairings_code = tcg.find('a', {'href': re.compile('/pairings*')})['href']

    roster_url = 'https://rk9.gg' + roster_code
    pairings_url = 'https://rk9.gg' + pairings_code
    
    return [roster_url,pairings_url]

In [99]:
def get_round_df(homepage, round_number):
    
    pairings_url = get_rk9_urls(homepage)[1]

    total_soup = BeautifulSoup(requests.get(pairings_url).text)
    total_rounds = int(total_soup.find_all('a', id = re.compile('P2R*'))[-2].text[1:])
    print('running round ' + str(round_number))
    pairing_soup = BeautifulSoup(requests.get(pairings_url, {'pod' : '2', 'rnd' : str(round_number)}).text)

    # Set up games as a list of the games played, and set a list of player 1 and player 2 for all games.

    games = pairing_soup.find_all('div', class_ = "row row-cols-3 match no-gutter complete")
    P1_names = [game.find('span', class_ = 'name').text for game in games]
    P2_names = [game.find_all('span', class_ = 'name')[-1].text for game in games]

    # Grab the match results, and deal with dropped players

    P1_result = [game.find('div', class_ = re.compile("col-5 text-center player*"))['class'][-1] for game in games]
    for i in range(len(P1_result)):
        game = games[i]
        if P1_result[i] == 'dropped':
            P1_result[i] = game.find('div', class_ = re.compile("col-5 text-center player*"))['class'][4]

        if P1_result[i] == 'dropped':
            P1_result[i] = 'double game loss'

    # Generate a DataFrame for the round

    round_dict = {
        'Player 1' : P1_names,
        'Player 2' : P2_names,
        'Result' : P1_result
        }

    round_df = pd.DataFrame(round_dict)

    # Remove players that didn't play

    mask = (round_df['Player 1'] == round_df['Player 2'])
    round_df = round_df[~mask]

    # Convert 'winner', 'loser', or 'tie' to 1, 0, or 0.5
    
    round_df.loc[round_df['Result'] == "loser",'Loser'] = round_df['Player 1']
    round_df.loc[round_df['Result'] == "winner",'Loser'] = round_df['Player 2']
    round_df.loc[round_df['Result'] == 'tie', 'Loser'] = 'Tie'

    round_df.loc[round_df['Result'] == "loser",'Result'] = round_df['Player 2']
    round_df.loc[round_df['Result'] == "winner",'Result'] = round_df['Player 1']
    round_df.loc[round_df['Result'] == 'tie', 'Result'] = 'Tie'

    return round_df

In [100]:
all_matchups = pd.DataFrame([],columns = ['Player 1', 'Player 2', 'Result'])

pairings_url = get_rk9_urls(homepage)[1]

total_soup = BeautifulSoup(requests.get(pairings_url).text)
total_rounds = int(total_soup.find_all('a', id = re.compile('P2R*'))[-2].text[1:])

for i in range(total_rounds):
    round_df = get_round_df(homepage, i+1)
    all_matchups = pd.concat([all_matchups,round_df],ignore_index = True)

running round 1
running round 2
running round 3
running round 4
running round 5
running round 6
running round 7
running round 8
running round 9
running round 10
running round 11
running round 12
running round 13
running round 14
running round 15


In [101]:
all_matchups

,Player 1,Player 2,Result,Loser
0,Giacomo Treppiedi [IT],Noemi Frasnelli [AT],Noemi Frasnelli [AT],Giacomo Treppiedi [IT]
1,Manuel Giuseppe La Iacona [IT],Mateusz Mikulski [IT],Mateusz Mikulski [IT],Manuel Giuseppe La Iacona [IT]
2,Claudio ippolito [IT],Elisa Piccolantonio [IT],Claudio ippolito [IT],Elisa Piccolantonio [IT]
3,Graziano Piccone [IT],Nathan VILLA [FR],Graziano Piccone [IT],Nathan VILLA [FR]
4,Tiziano Pacini [IT],Alexander Schlemmer [DE],Alexander Schlemmer [DE],Tiziano Pacini [IT]
...,...,...,...,...
4981,Fabrizio Tallone [IT],Francesco Paolo Palumbo [IT],Fabrizio Tallone [IT],Francesco Paolo Palumbo [IT]
4982,Riccardo Coviello [IT],Julius Brunfeldt [FI],Riccardo Coviello [IT],Julius Brunfeldt [FI]
4983,Angelo Coralluzzo [IT],Simon Tschan [CH],Simon Tschan [CH],Angelo Coralluzzo [IT]
4984,Fabrizio Tallone [IT],Riccardo Coviello [IT],Riccardo Coviello [IT],Fabrizio Tallone [IT]


In [102]:
all_matchups['Player 1'] = all_matchups['Player 1'].map(archetype_dict)
all_matchups['Player 2'] = all_matchups['Player 2'].map(archetype_dict)
all_matchups['Result'] = all_matchups['Result'].map(archetype_dict)
all_matchups['Loser Elo'] = all_matchups['Loser'].map(elo_dict)
all_matchups['Loser'] = all_matchups['Loser'].map(archetype_dict)

all_matchups

,Player 1,Player 2,Result,Loser,Loser Elo
0,Other,Straight Dragapult,Straight Dragapult,Other,952.061646
1,Other,Gardevoir ex,Gardevoir ex,Other,980.967962
2,Joltik Box,Raging Bolt ex,Joltik Box,Raging Bolt ex,964.437111
3,Other,Gholdengo ex,Other,Gholdengo ex,981.434258
4,Eevees,Tera box,Tera box,Eevees,964.811272
...,...,...,...,...,...
4981,Other,Dragapult Dusknoir,Other,Dragapult Dusknoir,1145.430121
4982,Gardevoir ex,Gardevoir ex,Gardevoir ex,Gardevoir ex,1354.508154
4983,Gardevoir ex,Other,Other,Gardevoir ex,1279.641403
4984,Other,Gardevoir ex,Gardevoir ex,Other,1262.791608


In [103]:
all_matchups.to_csv(tournament + '_all_matchups.csv')

In [104]:
all_jtg_matchups = pd.read_csv('all_jtg_matchups')
all_jtg_matchups = pd.concat([all_jtg_matchups, all_matchups], ignore_index=True)
all_jtg_matchups = all_jtg_matchups.drop(['Unnamed: 0'], axis=1)
#all_jtg_matchups.to_csv('all_jtg_matchups')

In [108]:
all_jtg_matchups

,Player 1,Player 2,Result,Loser,Loser Elo
0,Other,Dragapult Dusknoir,Other,Dragapult Dusknoir,973.817740
1,Gholdengo ex,Other,NaN,NaN,NaN
2,Dragapult Dusknoir,Other,Dragapult Dusknoir,Other,1028.151890
3,NaN,Gardevoir ex,NaN,Gardevoir ex,957.981883
4,Other,Tera box,Other,Tera box,1071.871805
...,...,...,...,...,...
43602,Other,Dragapult Dusknoir,Other,Dragapult Dusknoir,1145.430121
43603,Gardevoir ex,Gardevoir ex,Gardevoir ex,Gardevoir ex,1354.508154
43604,Gardevoir ex,Other,Other,Gardevoir ex,1279.641403
43605,Other,Gardevoir ex,Gardevoir ex,Other,1262.791608


In [109]:
all_jtg_matchups.to_csv('all_jtg_matchups.csv')

In [107]:
tournament

'pokemon-bologna-2025'